In [1]:
import os 


In [2]:
%pwd

'c:\\WORK\\Projects\\MLOps\\Project1\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\WORK\\Projects\\MLOps\\Project1'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.Project1.constants import *
from src.Project1.utils.common import read_yaml, create_dirs

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = config_file_path,
                 params_filepath = params_file_path,
                 schema_filepath = schema_file_path):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_dirs([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionconfig:
        config = self.config.data_ingestion
        create_dirs([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [8]:
from urllib.request import Request
import urllib.request as request
from src.Project1 import logger
import zipfile


In [9]:
# Data Ingestion component
class DataIngestion:
    def __init__(self, config:DataIngestionconfig):
        self.config =  config
      
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers  = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f" {filename} download! with following infow : \n {headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion  = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-28 14:19:08,982] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:31} INFO -  Read yaml file from config\config.yaml
[2025-03-28 14:19:08,985] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:31} INFO -  Read yaml file from params.yaml
[2025-03-28 14:19:08,987] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:31} INFO -  Read yaml file from schema.yaml
[2025-03-28 14:19:08,989] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:51} INFO - Created directory artifacts
[2025-03-28 14:19:08,990] {c:\WORK\Projects\MLOps\Project1\src\Project1\utils\common.py:51} INFO - Created directory artifacts/data_ingestion
[2025-03-28 14:19:10,722] {C:\Users\91720\AppData\Local\Temp\ipykernel_18216\1624680671.py:21} INFO -  artifacts/data_ingestion/data.zip download! with following infow : 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-T